In [25]:
pip install --upgrade pip

In [26]:
!pip install -U langchain-community


In [27]:
pip install cassio datasets langchain openai tiktoken

In [28]:
# LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper     # <-- to wrap all in specific package
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings           # <--  it will respnsible to convert your text into vector

# Support for dataset retrieval with Hugging Face
from datasets import load_dataset                           #  # <--  if we want some dataset we can use it

# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio                                                     #  # <--  it will help you to integrated with langchain


In [29]:
!pip install PyPDF2

In [30]:
from PyPDF2 import PdfReader

In [ ]:
ASTRA_DB_APPLICATION_TOKEN =, 
ASTRA_DB_ID =, 



In [32]:
pdfreader = PdfReader("/content/physics_chapter.pdf")

In [ ]:
# we are going to diveide all our content in some chunks

In [34]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):   # we are reading all our text data from all pages and extract all text
    content = page.extract_text()
    if content:
        raw_text += content

In [35]:
raw_text

"Chapter One\nELECTRIC CHARGES\nAND FIELDS\n1.1  I NTRODUCTION\nAll of us have the experience of seeing a spark or hearing a crackle when\nwe take of f our synthetic clothes or sweater , particularly in dry weather .\nHave you ever tried to find any explanation for this phenomenon? Another\ncommon example of electric discharge is the lightning that we see in the\nsky during thunderstor ms. W e also experience a sensation of an electric\nshock either while opening the door of a car or holding the iron bar of a\nbus after sliding from our seat. The reason for these experiences is\ndischarge of electric charges through our body, which were accumulated\ndue to rubbing of insulating sur faces. Y ou might have also hear d that\nthis is due to generation of static electricity. This is precisely the topic we\nare going to discuss in this and the next chapter . Static means anything\nthat does not move or change with time. Electrostatics deals with\nthe study of forces, fields and potentials ar

In [1]:
# iniiallize the connections with database by using casio.init

cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [37]:
# Now create Langcahin embedding for our data

llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [38]:
# Now we create Langcahin vector store of whcih is backed by AstraDB

# here we provide three im information
# 1) what kind of embedding we are using
# 2) what is the table name inside the database
# 3) session , keypace = none

astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)


In [ ]:
# but stil we not converted our data into vectore it happend when we push our data to vector

In [39]:
# now we convert our data into text chunks

from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it should not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
# texts = text_splitter.split_text(raw_text)   # <-- provide our data

In [40]:
# Assuming raw_text is defined elsewhere in your code
texts = text_splitter.split_text(raw_text)

# Clean the split texts by removing or replacing new line characters
cleaned_texts = [text.replace('\n', ' ') for text in texts]

# Check the cleaned split texts
for i, text in enumerate(cleaned_texts[:50]):
    print(f"Chunk {i+1}:\n{text}\n")

Chunk 1:
Chapter One ELECTRIC CHARGES AND FIELDS 1.1  I NTRODUCTION All of us have the experience of seeing a spark or hearing a crackle when we take of f our synthetic clothes or sweater , particularly in dry weather . Have you ever tried to find any explanation for this phenomenon? Another common example of electric discharge is the lightning that we see in the sky during thunderstor ms. W e also experience a sensation of an electric shock either while opening the door of a car or holding the iron bar of a bus after sliding from our seat. The reason for these experiences is discharge of electric charges through our body, which were accumulated due to rubbing of insulating sur faces. Y ou might have also hear d that this is due to generation of static electricity. This is precisely the topic we

Chunk 2:
due to rubbing of insulating sur faces. Y ou might have also hear d that this is due to generation of static electricity. This is precisely the topic we are going to discuss in this a

In [41]:
cleaned_texts

['Chapter One ELECTRIC CHARGES AND FIELDS 1.1  I NTRODUCTION All of us have the experience of seeing a spark or hearing a crackle when we take of f our synthetic clothes or sweater , particularly in dry weather . Have you ever tried to find any explanation for this phenomenon? Another common example of electric discharge is the lightning that we see in the sky during thunderstor ms. W e also experience a sensation of an electric shock either while opening the door of a car or holding the iron bar of a bus after sliding from our seat. The reason for these experiences is discharge of electric charges through our body, which were accumulated due to rubbing of insulating sur faces. Y ou might have also hear d that this is due to generation of static electricity. This is precisely the topic we',
 'due to rubbing of insulating sur faces. Y ou might have also hear d that this is due to generation of static electricity. This is precisely the topic we are going to discuss in this and the next c

In [42]:
# texts[:50]

cleaned_texts[:50]

['Chapter One ELECTRIC CHARGES AND FIELDS 1.1  I NTRODUCTION All of us have the experience of seeing a spark or hearing a crackle when we take of f our synthetic clothes or sweater , particularly in dry weather . Have you ever tried to find any explanation for this phenomenon? Another common example of electric discharge is the lightning that we see in the sky during thunderstor ms. W e also experience a sensation of an electric shock either while opening the door of a car or holding the iron bar of a bus after sliding from our seat. The reason for these experiences is discharge of electric charges through our body, which were accumulated due to rubbing of insulating sur faces. Y ou might have also hear d that this is due to generation of static electricity. This is precisely the topic we',
 'due to rubbing of insulating sur faces. Y ou might have also hear d that this is due to generation of static electricity. This is precisely the topic we are going to discuss in this and the next c

In [43]:
# we are taking top 50 data and store that in vectore database

# add the text inside the casandraDB

# here it will do embedding and also store the data inside astradb

In [44]:
# here we do embedding also and we store it also in astrdb

# astra_vector_store.add_texts(texts[:50])
astra_vector_store.add_texts(cleaned_texts[:50])

print("Inserted %i headlines." % len(cleaned_texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)          # we are going to wrap inside the wraper

## Now we have some  vector inside our data base now we can add/ ask our query

Inserted 50 headlines.


In [ ]:
## Now we have some  vector inside our data base now we can add/ ask our query

# WE can ask some of Question to it from our data so it can answer

In [46]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):    # K=4 means it is giving us top 4 result
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))  # it will give 84 charcat of text


Enter your question (or type 'quit' to exit): From which Greek word is the term "electricity" derived, and what does it mean?

QUESTION: "From which Greek word is the term "electricity" derived, and what does it mean?"


ANSWER: "The term "electricity" is derived from the Greek word "elektron," which means amber."

FIRST DOCUMENTS BY RELEVANCE:


    [0.9167] "due to rubbing of insulating sur faces. Y ou might have also hear d that this is due ..."
    [0.8986] "written as –e and that on a proton as +e. The fact that electric charge is always an ..."
    [0.8948] "created in the process of rubbing. Also the number of electrons, that are transferre ..."
    [0.8930] "bodies are brought in contact. What can you conclude from these observations? It jus ..."

What's your next question (or type 'quit' to exit): quite

QUESTION: "quite"


ANSWER: "I'm sorry, I couldn't find the word "quite" in the given context. Can you please provide more specific information or context for me to answer the question accurately?"

FIRST DOCUMENTS BY RELEVANCE:


    [0.8800] "2024-25 2024-25 2Physics elektr on meaning amber . Many such pairs of materials wer  ..."
    [0.8745] "we find  that (i) like charges repel  and (ii) unlike charges  attract  each other . ..."
    [0.8744] "rod attracts the glass rod [Fig. 1.1(c)] and repel the silk or wool with which the g ..."
    [0.8741] "bodies are brought in contact. What can you conclude from these observations? It jus ..."

What's your next question (or type 'quit' to exit): quit


Enter your question (or type 'quit' to exit): From which Greek word is the term "electricity" derived, and what does it mean?

QUESTION: "From which Greek word is the term "electricity" derived, and what does it mean?"
WARNING:cassandra.protocol:Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
ANSWER: "The term "electricity" is derived from the Greek word "elektron," which means amber."